In [3]:
/* Add Deps */
%AddDeps edu.stanford.nlp stanford-corenlp 3.7.0
%AddDeps com.google.protobuf protobuf-java 2.6.1
%AddDeps com.databricks spark-csv_2.10 1.5.0 --transitive

// Non-repo dependencies 
%AddJar file:lib/corenlp-models.jar
%AddJar file:SE/target/scala-2.10/se_2.10-1.1.jar

Marking edu.stanford.nlp:stanford-corenlp:3.7.0 for download
Preparing to fetch from:
-> file:/tmp/toree_add_deps7448866121703552186/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree_add_deps7448866121703552186/https/repo1.maven.org/maven2/edu/stanford/nlp/stanford-corenlp/3.7.0/stanford-corenlp-3.7.0.jar
Marking com.google.protobuf:protobuf-java:2.6.1 for download
Preparing to fetch from:
-> file:/tmp/toree_add_deps7448866121703552186/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree_add_deps7448866121703552186/https/repo1.maven.org/maven2/com/google/protobuf/protobuf-java/2.6.1/protobuf-java-2.6.1.jar
Marking com.databricks:spark-csv_2.10:1.5.0 for download
Preparing to fetch from:
-> file:/tmp/toree_add_deps7448866121703552186/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree_add_deps7448866121703552186/https/repo1.maven.org/maven2/org/apache/commons/commons-csv/1.1/commons-csv-1.1.jar
-> New file at /tmp/toree_add_deps7448866121703552186/https/repo1

In [4]:
import com.evan.kaggle.se.FeatureEngineering._
val sqlContext = org.apache.spark.sql.SQLContext.getOrCreate(sc)
import sqlContext.implicits._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import scala.collection.immutable.HashSet

In [5]:
val files = List("cooking")//, "crypto", "robotics", "biology", "travel", "diy")

/* Load and print all files */
val df_all = files.map(f => {
                        sqlContext.read.format("com.databricks.spark.csv").option("header", "true").load("../dat/"+f+"_clean.csv")
                   }).reduce(_ unionAll _)

In [16]:
val sub = df_all.sample(false, .1)
sub.show

+---+--------------------+--------------------+--------------------+
| id|               title|             content|                tags|
+---+--------------------+--------------------+--------------------+
| 17|How long and at w...|<p>I'm interested...|baking chicken co...|
| 81|How do I pound ch...|<p>Despite my bes...|chicken meat chic...|
|143|Why do my burgers...|<p>When I broil o...|meat hamburgers g...|
|183|Cutting boards: W...|<p>I have heard v...|equipment knives ...|
|184|What knives are '...|<p>I consider mys...|    equipment knives|
|190|Proper ratio of W...|<p>When making ri...|                rice|
|245|How can I keep me...|<p>Nothin' better...|baking eggs pie m...|
|337|Substitution for ...|<p>I'm interested...|substitutions bee...|
|352|Does it take a sp...|<p>Or am I just d...|equipment blender...|
|362|My Baba Ghanoush ...|<p>I tried cookin...|please-remove-thi...|
|386|What is the best ...|<p>What is the be...|storage-method fr...|
|399|What are the best...|<p>What 

In [17]:
val fiveGramFeats = makeTrFeatures(5) _
val featDF = sub.select("title", "content", "tags").
                 rdd.flatMap{case Row(t: String, c: String, ta: String) => 
                                                     fiveGramFeats(t, c, ta)}.toDF

In [18]:
featDF.show

+----------+-------+-----------+-------------------+--------+--------+-------+-----+
|     nGram|posTags|    depTags|             relPos|numWords|hasUpper|isTitle|isTag|
+----------+-------+-----------+-------------------+--------+--------+-------+-----+
|       <p>|     NN|       root|                0.0|       1|   false|  false|false|
|         I|    PRP|      nsubj|0.09090909090909091|       1|    true|  false|false|
|        be|    VBP|        cop|0.18181818181818182|       1|   false|  false|false|
|interested|     JJ|  acl:relcl| 0.2727272727272727|       1|   false|  false|false|
|        in|     IN|       case|0.36363636363636365|       1|   false|  false|false|
|      bake|    VBG|       amod|0.45454545454545453|       1|   false|  false| true|
|     thigh|    NNS|    nmod:in| 0.5454545454545454|       1|   false|  false|false|
|       leg|    NNS|   conj:and| 0.6363636363636364|       1|   false|  false|false|
|    breast|    NNS|   conj:and| 0.7272727272727273|       1|   f

In [19]:
val s = System.nanoTime
println(s"There are ${featDF.count} rows")
val diff = (System.nanoTime - s)/1e9d
println(s"Elapsed: $diff seconds")

There are 709075 rows
Elapsed: 72.573625535 seconds


In [20]:
featDF.count / df_all.count.toDouble

46.03187483770449

In [22]:
featDF.filter($"isTag").count

6225